In [1]:
%pip install import-ipynb


   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 2.1 MB/s eta 0:00:00


In [5]:
%pip install importnb

   ---------------------------------------- 0.0/46.0 kB ? eta -:--:--
   ---------------------------------------- 46.0/46.0 kB 760.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:

import import_ipynb
from importnb import Notebook

with Notebook():
    import solver

Note: you may need to restart the kernel to use updated packages.


In [9]:
#fancendo cosi utilizzando la classe PDSVRP_istance
#basterebbe solo questa cella di codice e poi chiamare la funzione test_solver_2

import PDSVRP_instance

def test_solver(instance_file_path):
    instance = PDSVRP_instance.PDSVRPInstance(instance_file_path)
    model = solver.PDSVRPModel(instance) 
    model.build_model()
    model.solve()
    model.print_results()

In [10]:
test_solver('instances/30-c-0-c.txt')

Set parameter Username


Academic license - for non-commercial use only - expires 2025-05-21
Set parameter Threads to value 8
Set parameter Presolve to value 2
Set parameter Cuts to value 3
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 8 threads

Optimize a model with 1026 rows, 2010 columns and 8412 nonzeros
Model fingerprint: 0xda6cee5e
Variable types: 960 continuous, 1050 integer (1050 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+03]
  Objective range  [6e-02, 3e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 161.5864695
Presolve removed 30 rows and 489 columns
Presolve time: 0.04s
Presolved: 996 rows, 1521 columns, 10386 nonzeros
Variable types: 495 continuous, 1026 integer (1026 binary)
Root relaxation presolve removed 1 rows and 0 columns
Root re

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\pelli\AppData\Roaming\Python\Python312\site-packages\ipykernel\iostream.py", line 626, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 3556971 2623783   43.14558   35   14   81.44335   42.28973  48.1%   7.4 2290s
 3561760 2627412   59.49265   66   11   81.44335   42.29079  48.1%   7.4 2296s
